# Ranking and Position Sizing

**PyBroker** includes support for ranking ticker symbols and setting position sizes on a group of symbols. This notebook goes over how to use both of these features in your trading strategy.

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('ranking_and_pos_sizing')

## Ranking Ticker Symbols

First, we will look at ranking ticker symbols when placing a buy order.

In [2]:
def buy_highest_volume(ctx):
    # If there are no long positions across all tickers being traded:
    if not tuple(ctx.long_positions()):
        ctx.buy_shares = ctx.calc_target_shares(1)
        ctx.hold_bars = 2
        ctx.score = ctx.volume[-1]

The above ranks ticker symbols by their most recent trading volume, allocating 100% of the portfolio for 2 days.

In [3]:
config = StrategyConfig(max_long_positions=1)
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022', config)
strategy.add_execution(buy_highest_volume, ['T', 'F', 'GM', 'PFE'])

When creating the [Strategy](https://pybroker.readthedocs.io/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy), we set [max_long_positions](https://pybroker.readthedocs.io/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.max_long_positions) to ```1```. This limits the number of long positions that can be held at any time to 1. The result is only placing a buy order for the ticker that has the highest volume from ```['T', 'F', 'GM', 'PFE']```:

In [4]:
result = strategy.backtest(calc_bootstrap=False)
result.orders

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

[*********************100%***********************]  4 of 4 completed
Finished download: 0:00:02 

Test split: 2021-06-01 04:00:00 to 2022-05-31 04:00:00


100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:04


,date,symbol,order_type,limit_price,fill_price,shares,pnl,pnl %
id,,,,,,,,
1,2021-06-02 04:00:00,F,buy,NaN,14.85,6734,0.00,0.000000
2,2021-06-04 04:00:00,F,sell,NaN,16.13,6734,8619.52,99.813216
3,2021-06-07 04:00:00,F,buy,NaN,15.93,6801,0.00,0.000000
4,2021-06-09 04:00:00,F,sell,NaN,15.51,6801,-2856.42,-99.459945
5,2021-06-10 04:00:00,F,buy,NaN,15.43,6832,0.00,0.000000
...,...,...,...,...,...,...,...,...
164,2022-05-20 04:00:00,F,sell,NaN,12.59,6739,-2965.16,-99.577198
165,2022-05-23 04:00:00,F,buy,NaN,12.72,6936,0.00,0.000000
166,2022-05-25 04:00:00,F,sell,NaN,12.57,6936,-1040.40,-98.806234


## Setting Position Sizes

It is also possible to set position sizes based on multiple tickers. Let's start with a simple buy and hold strategy that begins trading after 100 days, and holds the position for 30 days.

In [5]:
def buy_and_hold(ctx):
    if not ctx.long_pos() and len(ctx.close) > 100:
        ctx.buy_shares = 100
        ctx.hold_bars = 30
        
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022')
strategy.add_execution(buy_and_hold, ['T', 'F', 'GM', 'PFE'])

The above will buy ```100``` shares in each of ```['T', 'F', 'GM', 'PFE']```. But what if we do not want to use equal position sizing? For example, we may want to size positions so that more shares are allocated to tickers with lower volatility to try to decrease the portfolio's overall volatility.

We define a function that does just that below:

In [6]:
import numpy as np

def pos_size_handler(ctx):
    # Fetch all buy signals.
    signals = tuple(ctx.signals("buy"))
    # Return if there are no buy signals (i.e. there are only sell signals).
    if not signals:
        return
    # Calculates the inverse volatility, where volatility is defined as the
    # standard deviation of close prices for the last 100 days.
    get_inverse_volatility = lambda signal: 1 / np.std(signal.bar_data.close[-100:])
    # Sums the inverse volatilities for all of the buy signals.
    total_inverse_volatility = sum(map(get_inverse_volatility, signals))
    for signal in signals:
        size = get_inverse_volatility(signal) / total_inverse_volatility
        # Calculate the number of shares given the latest close price.
        shares = ctx.calc_target_shares(size, signal.bar_data.close[-1])
        ctx.set_shares(signal, shares)
        
strategy.set_pos_size_handler(pos_size_handler)

The [pos_size_handler](https://pybroker.readthedocs.io/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_pos_size_handler) will run on every bar that generates a buy or sell signal when [buy_shares](https://pybroker.readthedocs.io/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_shares) or [sell_shares](https://pybroker.readthedocs.io/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.sell_shares) is set on the [ExecContext](https://pybroker.readthedocs.io/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext).

In [7]:
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loaded cached bar data.

Test split: 2021-06-01 04:00:00 to 2022-05-31 04:00:00


100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


In [8]:
result.orders

,date,symbol,order_type,limit_price,fill_price,shares,pnl,pnl %
id,,,,,,,,
1,2021-10-22 04:00:00,PFE,buy,NaN,42.91,251,0.00,0.000000
2,2021-10-22 04:00:00,GM,buy,NaN,58.17,140,0.00,0.000000
3,2021-10-22 04:00:00,F,buy,NaN,16.39,2150,0.00,0.000000
4,2021-10-22 04:00:00,T,buy,NaN,19.36,2368,0.00,0.000000
5,2021-12-06 05:00:00,PFE,sell,NaN,52.57,251,2424.66,97.877872
6,2021-12-06 05:00:00,GM,sell,NaN,59.72,140,217.00,78.418618
7,2021-12-06 05:00:00,F,sell,NaN,19.05,2150,5719.00,99.668006
8,2021-12-06 05:00:00,T,sell,NaN,17.81,2368,-3670.40,-99.517110
9,2021-12-07 05:00:00,PFE,buy,NaN,51.45,321,0.00,0.000000


Setting position sizes in this manner opens up a lot of possibilities, such as using [Mean-Variance Optimization](https://en.wikipedia.org/wiki/Modern_portfolio_theory) to determine portfolio allocations!

[In the next notebook, we will switch gears and discuss how to implement custom indicators in **PyBroker**.](https://pybroker.readthedocs.io/en/latest/notebooks/5.%20Writing%20Indicators.html)